In [5]:
#!pip install findspark
#!pip install pandas
#!pip install pyspark

                                              0.0/316.9 MB ? eta -:--:--
                                              0.0/316.9 MB 1.9 MB/s eta 0:02:46
                                              0.2/316.9 MB 1.8 MB/s eta 0:02:55
                                              0.3/316.9 MB 2.2 MB/s eta 0:02:24
                                              0.5/316.9 MB 2.6 MB/s eta 0:02:01
                                              0.5/316.9 MB 2.6 MB/s eta 0:02:02
                                              0.7/316.9 MB 2.7 MB/s eta 0:01:58
                                              0.8/316.9 MB 2.6 MB/s eta 0:02:00
                                              1.0/316.9 MB 2.9 MB/s eta 0:01:48
                                              1.2/316.9 MB 2.9 MB/s eta 0:01:49
                                              1.3/316.9 MB 2.9 MB/s eta 0:01:49
                                              1.5/316.9 MB 2.9 MB/s eta 0:01:49
                                              1

ERROR: Exception:
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\ProgramData\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 90, in read
    data = self.__fp.read(amt)
           ^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\http\client.py", line 466, in read
    s = self.fp.read(amt)
        ^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^

In [68]:
import pyspark

#Crear una sesión de Spark
from pyspark.sql import SparkSession

#Importar funciones y tipos
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType, IntegerType

#Crear el objeto Spark session
spark = SparkSession.builder.appName("BD12").getOrCreate()


In [113]:
##### CARGO LOS DATASETS

#Leer el archivo de disco --------------->> SE DEBE AJUSTAR LA RUTA

dfOriginal2021 = spark.read.option("header","true").csv("C:/Datasets/world-happiness-report-2021.csv")

dfOriginalAll = spark.read.option("header","true").csv("C:/Datasets/world-happiness-report.csv")

#dfOriginal2021.printSchema()

In [26]:
dfWork1 = dfOriginal2021.select(col("Country name").alias("CountryName"),
                            col("Regional indicator").alias("RegionalIndicator"),
                            col("Ladder score").cast(DoubleType()).alias("LadderScore"))
#dfWork1.printSchema()

root
 |-- CountryName: string (nullable = true)
 |-- RegionalIndicator: string (nullable = true)
 |-- LadderScore: double (nullable = true)



In [ ]:
#
# Ejercicio 1 - ¿Cuál es el país más “feliz” del 2021 según la data? (considerar que la columna “Ladder score” 
#               mayor número más feliz es el país)

In [22]:
dfPunto1 = dfWork1.orderBy(col("LadderScore").desc()).first()

print("\033[1mEl pais con mayor indice de felicidad en 2021 es:\033[0m")
print("Pais  :", dfPunto1["CountryName"])
print("Score :", dfPunto1["LadderScore"])


El pais con mayor indice de felicidad en 2021 es
Pais  : Finland
Score : 7.842


In [ ]:
#
# Ejercicio 2 - ¿Cuál es el país más “feliz” del 2021 por continente según la data
#

In [56]:
from pyspark.sql.window import Window
from pyspark.sql import functions as f

dfPunto2 = dfWork1.groupBy("RegionalIndicator", "CountryName").agg(f.max("LadderScore").alias("MaxLadderScore"))

window_spec = Window.partitionBy("RegionalIndicator")
max_ladder_score_df = dfPunto2.withColumn("LadderScore", f.max("MaxLadderScore").over(window_spec ))

#max_ladder_df.show()
                               
dfPunto2 = max_ladder_score_df.filter(f.col("LadderScore") == f.col("MaxLadderScore"))
        
dfPunto2 = dfPunto2.select("RegionalIndicator", "CountryName", "LadderScore")

#Seleccionar las columans
print("\033[1mLos paises mas felices por continente del 2021 son:\033[0m")

dfPunto2 = dfPunto2.withColumnRenamed("RegionalIndicator", "Regional Indicator") \
                    .withColumnRenamed("CountryName", "Country Name") \
                    .withColumnRenamed("LadderScore", "Ladder Score")

display(dfPunto2.toPandas())


Los paises mas felices por continente del 2021 son:


,Regional Indicator,Country Name,Ladder Score
0,Central and Eastern Europe,Czech Republic,6.965
1,Commonwealth of Independent States,Uzbekistan,6.179
2,East Asia,Taiwan Province of China,6.584
3,Latin America and Caribbean,Costa Rica,7.069
4,Middle East and North Africa,Israel,7.157
5,North America and ANZ,New Zealand,7.277
6,South Asia,Nepal,5.269
7,Southeast Asia,Singapore,6.377
8,Sub-Saharan Africa,Mauritius,6.049
9,Western Europe,Finland,7.842


In [ ]:
#
# Ejercicio 3 - ¿Cuál es el país que más veces ocupó el primer lugar en todos los años?
#

In [199]:
import pyspark.sql.functions as F
from pyspark.sql.window import Window


#Selecciono las columnas a utilizar
dfWork3 = dfOriginalAll.select( col("Country name").alias("CountryName"),
                                col("year").cast(IntegerType()).alias("Year"), 
                                col("Life Ladder").cast(DoubleType()).alias("LifeLadder"))

# Creo las particiones para numerar las posiciones
WinYear = Window.partitionBy("Year").orderBy(F.desc("LifeLadder"))
dfWork3 = dfWork3.withColumn("Ranking", F.row_number().over(WinYear))

# Selecciono los paises que tuvvieron la posicion 1
dfPosiciones1 = dfWork3.filter(F.col("Ranking") == 1)

# Agrupo y cuento las posiciones 1 que tuvo cada pais
dfResultado = dfPosicion1.groupBy("CountryName").agg(F.count("*").alias("Ranking1Count"))

# Ordeno el resultado descendente.
dfResultado = result_df.orderBy(F.desc("Ranking1Count"))

# Selecciono el pais con mas veces la posicion 1
Mas_Frecuente_Pos1 = dfResultado.first()




print("\033[1mPaís con más veces el ranking de la posición #1 es:\033[0m", most_frequent_ranking1["CountryName"])




País con más veces el ranking de la posición 1 es: Denmark


In [ ]:
#
# Ejercicio 4 - ¿Qué puesto de Felicidad tiene el país con mayor GDP del 2020?
#

In [208]:
#Selecciono las columnas a utilizar
dfWork4 = dfOriginalAll.select( col("Country name").alias("CountryName"),
                                col("year").cast(IntegerType()).alias("Year"), 
                                col("Life Ladder").cast(DoubleType()).alias("LifeLadder"),
                                col("Log GDP per capita").cast(DoubleType()).alias("GDP"))

# Creo las particiones para numerar las posiciones
WinYear = Window.partitionBy("Year").orderBy(F.desc("LifeLadder"))
dfWork4 = dfWork4.withColumn("Ranking", F.row_number().over(WinYear))

#dfWork4.show()

dfResultado = dfWork4.filter(F.col("Year") == 2020).orderBy(F.desc("GDP"))

Puesto_Felicidad_Pais_Mayor_GDP_2020 = dfResultado.first()


print("\033[1mEl país con mayor GDP en 2020 es\033[0m", Puesto_Felicidad_Pais_Mayor_GDP_2020["CountryName"] , " \033[1my ocupa la posición de Felicidad :\033[0m", Puesto_Felicidad_Pais_Mayor_GDP_2020["Ranking"])


El país con mayor GDP en 2020 es Ireland  y ocupa la posición de Felicidad : 13


In [ ]:
#
# Ejercicio 5 - ¿En que porcentaje a variado a nivel mundial el GDP promedio del 2020 respecto al 2021? ¿Aumentó 
#                o disminuyó?

In [ ]:
#
# Ejercicio 6 - ¿Cuál es el país con mayor expectativa de vide (“Healthy life expectancy at birth”)? Y ¿Cuánto tenia 
#                en ese indicador en el 2019?
